# Building Multi-Agent Systems with AutoGen

In this notebook, we'll explore how to build intelligent agents using the AutoGen framework. We'll start with the basics and progress to building a multi-agent system.

## What is AutoGen?

AutoGen is a framework that enables the development of LLM applications using multiple agents that can converse with each other to solve tasks. AutoGen agents can:
- Process and respond to user queries
- Use tools (functions) to interact with external systems
- Maintain context through conversation
- Be orchestrated to work together on complex tasks

Let's start by setting up our environment.

In [ ]:
# Install required packages if needed
# !pip install autogen-agentchat autogen-ext python-dotenv pytz

In [ ]:
# Import necessary libraries
import asyncio
import os
import pytz
from datetime import datetime
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import MagenticOneGroupChat
from autogen_agentchat.teams._group_chat._magentic_one._magentic_one_orchestrator import MagenticOneOrchestrator
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient, AzureOpenAIChatCompletionClient
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

## 1. Setting Up Azure OpenAI

To use agents with AutoGen, we need to set up a connection to an LLM service. In this example, we'll use GitHub Models, which gives us easy access to the latest models.

In [ ]:
if os.environ.get("GITHUB_TOKEN") is None:
    model_client = AzureOpenAIChatCompletionClient(
        azure_deployment=os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME"),
        model=os.getenv("AZURE_OPENAI_COMPLETION_MODEL"),
        api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
        model_info={
            "json_output": True,
            "function_calling": True,
            "vision": False,
            "family": "unknown",
        },
    )
else:
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings. 
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    token = os.environ["GITHUB_TOKEN"]
    endpoint = "https://models.inference.ai.azure.com"
    model_name = "gpt-4o-mini"

    # Create a model client for AutoGen
    model_client = OpenAIChatCompletionClient(
        model=model_name,
        base_url=endpoint,
        api_key=token
    )

## 2. Creating Tools

In AutoGen, tools are functions that agents can use to perform tasks. Let's define several tools that our agents can use:

In [ ]:
# Define tools for our agents
async def get_weather(city: str) -> str:
    """Gets a statement about the current weathr in the city defined in the parameter"""
    print("executing get_weather")
    return f"The weather in {city} is 73 degrees and Sunny."

async def get_medical_history(username: str) -> str:
    "Get the medical history for a given username with known allergies and food restrictions."
    print("executing get_medical_history")
    return f"{username} has an allergy to peanuts and eggs."

async def get_available_incredients(location: str) -> str:
    "Get the available incredients for a given location."
    print("executing get_available_incredients")
    return f"Available incredients in {location} are: eggs, milk, bread, peanuts, beer, wine, salmon, spinache, oil and butter."

def get_current_username(input: str) -> str:
    "Get the username of the current user."
    print("executing get_current_username")
    return "Dennis"

def get_current_location_of_user(username: str) -> str:
    "Get the current timezone location of the user for a given username."
    print("executing get_current_location")
    print(username)
    if "Dennis" in username:
        return "Europe/Berlin"
    else:
        return "America/New_York"

def get_current_time(location: str) -> str:
    "Get the current time in the given location. The pytz is used to get the timezone for that location. Location names should be in a format like America/Seattle, Asia/Bangkok, Europe/London. Anything in Germany should be Europe/Berlin"
    try:
        print("get current time for location: ", location)
        timezone = pytz.timezone(location)
        # Get the current time in the timezone
        now = datetime.now(timezone)
        current_time = now.strftime("%I:%M:%S %p")
        return current_time
    except Exception as e:
        print("Error: ", e)
        return "Sorry, I couldn't find the timezone for that location."
    

# Test one of our tools
await get_weather("Seattle")

## 3. Creating Specialized Agents

Now let's create specialized agents, each with their own set of tools and responsibilities:

In [ ]:
users_agent = AssistantAgent(
    "users_agent",
    model_client=model_client,
    tools=[get_current_username, get_medical_history],
    description="A helpful assistant that can knows things about the user like the username and the medical history of the user.",
    system_message="You are a helpful assistant that can retrieve the username and medical history of the current user.",
)

location_agent = AssistantAgent(
    "location_agent",
    model_client=model_client,
    tools=[get_current_location_of_user],
    description="A assistant that can find the physical location of a user.",
    system_message="You are a helpful assistant that can suggest details for a location and can utilize any context information provided.",
)

time_agent = AssistantAgent(
    "time_agent",
    model_client=model_client,
    tools=[get_current_time],
    description="A helpful assistant that knows time in a specific location.",
    system_message="You are a helpful assistant that can retrieve the current time for a given location.",
)

chef_agent = AssistantAgent(
    "chef_agent",
    model_client=model_client,
    tools=[get_available_incredients],
    description="A helpful assistant that can suggest meals and dishes for the right time of the day, location, available ingredients, user preferences and allergies.",
    system_message="You are a helpful assistant that can recommend dishes for the right time of the day, location, available ingredients and user preferences. Make sure you ask for individual food preferences and allergies as input. If you do not have concrete information about allergies you must ask the question and not prepare a dish until you get information.",
)

## 4. Creating a Summary Agent

In addition to our specialized agents, we'll create a summary agent that can synthesize the information from all other agents and provide a final response:

In [ ]:
# Create a summary agent
summary_agent = AssistantAgent(
    "summary_agent",
    model_client=model_client,
    description="A helpful assistant that can summarize details about conversations.",
    system_message="You are a helpful assistant that can take in all of the suggestions and advice from the other agents and leverage them to answer questions. You must ensure that you use the other agents. If you are done you respond with TERMINATE.",
)

## 5. Running a Multi-Agent Conversation

Now that we have created our agents, let's put them to work together in a multi-agent conversation. In AutoGen, we use group chat implementations to orchestrate the conversation between agents:

In [ ]:
async def run_multi_agent_conversation():
    # Set up termination condition - limit to 10 messages to avoid infinite loops
    inner_termination = MaxMessageTermination(10)
    # Set up termination condition - look for the word "TERMINATE" in agent messages
    termination_condition = TextMentionTermination("TERMINATE")

    combined_termination = inner_termination | termination_condition
    
    # Create a Magentic One group chat with our specialized agents
    # This is a type of group chat where agents can collaborate based on their specialized knowledge
    magenticteam = MagenticOneGroupChat(
        [users_agent, location_agent, time_agent, chef_agent, summary_agent], 
        model_client=model_client, 
        termination_condition=combined_termination,
        final_answer_prompt="Don't ask the user for anything else. Just provide the final answer."
    )

    # Run the team and stream messages to the console
    stream = magenticteam.run_stream(task="I want to have something to eat. What would you recommend?")
    await Console(stream)

# Run our multi-agent conversation
await run_multi_agent_conversation()

## 6. Understanding MagenticOneGroupChat

The `MagenticOneGroupChat` is a specific type of group chat in AutoGen that uses a special orchestration method to determine which agent should respond next. This is different from the `RoundRobinGroupChat`, which would cycle through the agents in order.

In [ ]:
# Example of using a round robin chat (alternative to MagenticOne)
async def run_round_robin_chat():
    # Set up termination condition - limit to 10 messages to avoid infinite loops
    inner_termination = MaxMessageTermination(10)
    # Set up termination condition - look for the word "TERMINATE" in agent messages
    termination_condition = TextMentionTermination("TERMINATE")
    
    # Create a round robin chat where agents take turns in order
    # We're adding the summary agent to help bring the conversation to a conclusion
    round_robin_team = RoundRobinGroupChat(
        [users_agent, location_agent, time_agent, chef_agent, summary_agent],
        termination_condition=termination_condition
    )
    
    # Run the team with a different task
    stream = round_robin_team.run_stream(task="I'm hungry but I need to consider my dietary restrictions. What should I eat?")
    await Console(stream)

await run_round_robin_chat()

## 7. Exercise: Create Your Own Multi-Agent System

Now it's your turn! Try creating your own multi-agent system with AutoGen. Here are some ideas:
- A travel planning system with agents for flights, hotels, activities, and weather
- A research assistant system with agents for searching, summarizing, and fact-checking
- A customer support system with agents for different types of issues

Here's a template to get you started:

In [ ]:
# Create your own tools
async def my_custom_tool(parameter: str) -> str:
    """Description of what your tool does"""
    print(f"Executing my_custom_tool with {parameter}")
    # Your tool implementation here
    return f"Result for {parameter}"

# Create your own agents
my_agent = AssistantAgent(
    "my_agent",
    model_client=model_client,
    tools=[my_custom_tool],
    description="Description of what your agent does",
    system_message="Detailed instructions for your agent",
)

# Create a group chat with your agents
# my_team = MagenticOneGroupChat([my_agent, ...], model_client=model_client)

# Run your team
# stream = my_team.run_stream(task="Your task here")
# await Console(stream)

## 8. Conclusion

In this notebook, we've explored how to build intelligent multi-agent systems using AutoGen:

1. We set up a connection to Azure OpenAI (GitHub Models) for the LLM backend.
2. We created tools that agents can use to interact with external systems.
3. We built specialized agents with clearly defined roles and tools.
4. We orchestrated multiple agents to collaboratively solve a task using group chats.

AutoGen provides a powerful framework for creating advanced multi-agent systems that can solve complex tasks through the collaboration of specialized agents. This approach allows for more robust and capable AI systems that can leverage the strengths of different types of agents working together.

For more information, visit the [AutoGen documentation](https://microsoft.github.io/autogen/).